In [1]:
# Scripts
from models import Base, Authors, Books, Stores, Inventory, BookAuthors, Customers, Orders, OrderDetails, Reviews, AuthorOverview, author_overview_def
from csv_data import read_upload
from db import get_engine, server_connection

# Libraries
from datetime import date, datetime
from typing import Optional, List
from sqlalchemy import create_engine, text, Integer, BigInteger, String, ForeignKey, Date, CheckConstraint, select, func, case, text, cast, DDL, event
from sqlalchemy.engine import URL
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, Session
from sqlalchemy.exc import ProgrammingError
import pandas as pd


# Setting up a server connection and creating a database

server_name = "Alstation"
database_name = "bookstore"
username = "sa"
password = "D@tabases"

server_engine = get_engine(server_name, username, password)
server_connection(server_engine, database_name)
engine = get_engine(server_name, username, password, database_name)
print(f"\nConnection to {database_name} successful!")


# Create and fill tables

Base.metadata.create_all(engine)
read_upload(Base, engine)


# Access the author overview

with engine.connect() as conn:
    conn.execute(author_overview_def)
    conn.commit()
session = Session(engine)
results = session.query(AuthorOverview).all()
for author in results:
    print(f"""
        {str(author.name).ljust(15)}
        {str(author.age).ljust(15)}
        {str(author.titles).ljust(15)}
        {str(author.inventory).ljust(15)}
    """)




Connection to bookstore successful!


ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]'CREATE VIEW' must be the first statement in a query batch. (111) (SQLExecDirectW)")
[SQL: 
    if object_id('author_overview', 'V') is not null drop view author_overview
    create view author_overview as
    select 
        a.id,
        concat(a.first_name, ' ', a.last_name) as name,
        cast(datediff(year, a.birth_date, cast(getdate() AS date)) as nvarchar(max)) + ' years' as age,
        cast(count(distinct b.isbn) as nvarchar(max)) + ' copies' as titles,
        cast(sum(i.amount * b.price) as nvarchar(max)) + ' SEK' as inventory
    from 
        authors a 
    join
        book_authors ba on a.id = ba.author_id
    join
        books b on ba.isbn = b.isbn
    join 
        inventory i on b.isbn = i.isbn
    group by
        a.id,
        a.first_name,
        a.last_name,
        a.birth_date
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
# engine.dispose()  # now the database is no longer in use